# MNIST & CNN with Visdom - Visdom을 이용해 Cost 시각화하기

## Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
import visdom

vis = visdom.Visdom()
vis.close(env='main')

Setting up a new session...


''

## Define loss_tracker

In [3]:
def loss_tracker(loss_plot, loss_value, num):
  '''num, loss_value are Tensor'''
  vis.line(X=num,
           Y=loss_value,
           win=loss_plot,
           update='append'
           )

## Device & reproducibility


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

print(device)

cuda


## Parameters

In [5]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# Data & Data Loader


In [6]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download= True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download= True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size= batch_size,
                                          shuffle= True,
                                          drop_last= True)

# Model

In [7]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size= 3, stride =1, padding= 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2, stride= 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size= 3, stride= 1,padding= 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2, stride= 2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size= 3, stride= 1, padding= 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2, stride= 2)
    )

    self.fc1 = nn.Linear(3*3*128, 625, bias=True)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(625, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)

    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [8]:
model = CNN().to(device)

value = torch.Tensor(1, 1, 28, 28).to(device)
print( (model(value)).shape )

torch.Size([1, 10])


## Cost function & optimizer

In [9]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

## make plot

In [10]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='Loss Tracker',
                                                        legend=['loss'],
                                                        showlegend= True))

## Training

In [11]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    
    avg_cost += cost / total_batch
  
  print('Epoch: {:2d}  Cost= {}'.format(epoch+1, avg_cost))
  loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished')

Epoch: 1  Cost= 0.1611432284116745
Epoch: 2  Cost= 0.043487902730703354
Epoch: 3  Cost= 0.030695373192429543
Epoch: 4  Cost= 0.021754659712314606
Epoch: 5  Cost= 0.017906855791807175
Epoch: 6  Cost= 0.014571031555533409
Epoch: 7  Cost= 0.013005176559090614
Epoch: 8  Cost= 0.010678024031221867
Epoch: 9  Cost= 0.007929728366434574
Epoch: 10  Cost= 0.008848415687680244
Epoch: 11  Cost= 0.008816732093691826
Epoch: 12  Cost= 0.006029883865267038
Epoch: 13  Cost= 0.006057250313460827
Epoch: 14  Cost= 0.00798111129552126
Epoch: 15  Cost= 0.005269593093544245
Learning Finished


## Testing

In [12]:
with torch.no_grad():
  X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)  
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

Accuracy: 0.9902999997138977
